# Long read assembly stats & BUSCO

## 1. Filter contigs < 1000 bases

In [1]:
WORKDIR=/workspace/hraczw/github/GA/Gillenia_genome/100.assemblies

In [3]:
# filter short seqs from assemblies

python << EOF

import sys, os

filename = '$WORKDIR/assemblyFileNames.txt'
f = open(filename,'r')

for line in f:
    sampleName = line.split('.fasta')[0]
    os.system('bsub -J preseq \
               -o $WORKDIR/' + sampleName + '_preseq.out \
               -e $WORKDIR/' + sampleName + '_preseq.err \
               "perl /workspace/hraczw/github/programs/prinseq-lite-0.20.4/prinseq-lite.pl \
               -fasta $WORKDIR/' + sampleName + '.fasta \
               -out_good $WORKDIR/' + sampleName + '.ml1000 \
               -out_bad $WORKDIR/' + sampleName + '.short \
               -min_len 1000"')

f.close()

EOF

Job <374708> is submitted to default queue <normal>.
Job <374709> is submitted to default queue <normal>.


## 2. Assembly stats

In [4]:
module load assemblathon_stats/14dfdab

In [5]:
python << EOF

import sys, os

filename = '$WORKDIR/assemblyFileNames.txt'
f = open(filename,'r')

for line in f:
    sampleName = line.split('.fasta')[0]
    os.system('bsub -J stats \
               -o $WORKDIR/' + sampleName + '_stats.ml1000.out \
               -e $WORKDIR/' + sampleName + '_stats.ml1000.err \
               "assemblathon_stats.pl $WORKDIR/' + sampleName + '.ml1000.fasta"')

f.close()

EOF

Job <374710> is submitted to default queue <normal>.
Job <374711> is submitted to default queue <normal>.


In [6]:
python << EOF

import sys, os

filename = '$WORKDIR/assemblyFileNames.txt'
f = open(filename,'r')
f1 = open('$WORKDIR/assemblyStats.ml1000.txt','w')

for line in f:
    sampleName = line.split('.fasta')[0]
    statsFile = open('$WORKDIR/' + sampleName + '_stats.ml1000.out','r')
#    print ('stats for ' + sampleName)
    print (sampleName + '\t')
    f1.write(sampleName + '\t')
    for line1 in statsFile:
        if 'Number of scaffolds' in line1 and '>' not in line1:
            print (line1.split('\n')[0])
            f1.write(line1.split('\n')[0] + '\t')
        if 'Total size of scaffolds' in line1:
            print (line1.split('\n')[0])
            f1.write(line1.split('\n')[0] + '\t')
        if 'Longest scaffold' in line1:
            print (line1.split('\n')[0])
            f1.write(line1.split('\n')[0] + '\t')            
        if 'Shortest scaffold' in line1:
            print (line1.split('\n')[0])
            f1.write(line1.split('\n')[0] + '\t')
        if 'Number of scaffolds > 10K nt' in line1:
            print (line1.split('\n')[0])
            f1.write(line1.split('\n')[0] + '\t')
        if 'Number of scaffolds > 100K nt' in line1:
            print (line1.split('\n')[0])
            f1.write(line1.split('\n')[0] + '\t')        
        if 'Number of scaffolds > 1M nt' in line1:
            print (line1.split('\n')[0])
            f1.write(line1.split('\n')[0] + '\t')
        if 'N50 scaffold length' in line1:
            print (line1.split('\n')[0])
            f1.write(line1.split('\n')[0] + '\t')
        if 'scaffold %N' in line1:
            print (line1)
            f1.write(line1 + '\n')
    print ('\n')
    statsFile.close()
f1.close()
f.close()

EOF

Flye_all_i1.assembly	
                                         Number of scaffolds       2184
                                     Total size of scaffolds  315818528
                                            Longest scaffold   12730551
                                           Shortest scaffold       1022
                                Number of scaffolds > 10K nt       1614  73.9%
                               Number of scaffolds > 100K nt        636  29.1%
                                 Number of scaffolds > 1M nt         42   1.9%
                                         N50 scaffold length     511599
                                                 scaffold %N       0.00



Flye_all_trimmed_i1.assembly	
                                         Number of scaffolds       2167
                                     Total size of scaffolds  315804247
                                            Longest scaffold    8832208
                                           Shortest scaffold

## 3. BUSCO

In [7]:
mkdir $WORKDIR/BUSCO

In [8]:
cp /workspace/hraczw/github/GA/Bilberry_genome/100.all.assemblies/BUSCO/config.ini $WORKDIR/BUSCO

In [9]:
export BUSCO_CONFIG_FILE=$WORKDIR/BUSCO/config.ini

In [11]:
module unload perl
module load BUSCO
module list

Currently Loaded Modulefiles:
  1) powerPlant/core             11) cegma/v2.5
  2) texlive/20151117            12) assemblathon_stats/14dfdab
  3) pandoc/1.19.2               13) pfr-python3/3.6.1
  4) git/2.21.0                  14) boost/1_61_0
  5) perlbrew/0.76               15) bamtools/2.4.0
  6) asub/2.1                    16) perl/bio-extras
  7) geneid/v1.4.4               17) augustus/3.2.2
  8) wise/2.4.1                  18) emboss/6.5.7
  9) hmmer/3.1b2                 19) BUSCO/v3.0.2
 10) ncbi-blast/2.2.25


In [12]:
export AUGUSTUS_CONFIG_PATH=/workspace/hraczw/github/GA/Bilberry_genome/100.all.assemblies/BUSCO/config

In [ ]:
# if it is busco v5
# export path incorperating with singularity

export SINGULARITYENV_AUGUSTUS_CONFIG_PATH=/workspace/hraczw/scriptomics/BUSCO/config/

In [15]:
python << EOF

import sys, os

sampleFile = open('$WORKDIR/assemblyFileNames.txt','r')

for line in sampleFile:
    sampleName = line.split('.fasta')[0]
    os.system('mkdir $WORKDIR/BUSCO/' + sampleName + '_tmp')
    os.system('bsub -J BUSCO \
               -n 12 \
               -o $WORKDIR/BUSCO/' + sampleName + '_BUSCO.out \
               -e $WORKDIR/BUSCO/' + sampleName + '_BUSCO.err \
               "run_BUSCO.py \
                -i $WORKDIR/' + sampleName + '.fasta \
                -l /workspace/hraczw/github/programs/embryophyta_odb10 \
                -f \
                -c 12 \
                -o ' + sampleName + '_BUSCO \
                -t $WORKDIR/BUSCO/' + sampleName + '_tmp \
                -m genome"')

sampleFile.close()

EOF

Job <374717> is submitted to default queue <normal>.
Job <374718> is submitted to default queue <normal>.


In [ ]:
python << EOF

import sys, os

sampleFile = open('$WORKDIR/assemblyFileNames.txt','r')
statsFile = open('$WORKDIR/BUSCO/BUSCO_stats.txt','w')

print ('Complete\tSingle\tDuplicate\n')
statsFile.write('SampleID\tComplete\tSingle\tDuplicate\tFragmented\tMissing\n')
for line in sampleFile:
    sampleName = line.split('.fasta')[0]
    print (sampleName + '\t')
    statsFile.write(sampleName + '\t')
    BUSCO_result = open('$WORKDIR/BUSCO/run_' + sampleName + '_BUSCO/short_summary_' + sampleName + '_BUSCO.txt','r')
    for line1 in BUSCO_result:
        if 'C:' in line1:
            print (line1.split('C:')[1].split('[')[0] + '\t')
            print (line1.split('[S:')[1].split(',')[0] + '\t')
            print (line1.split('D:')[1].split(']')[0] + '\t')
            print (line1.split(',F:')[1].split(',')[0] + '\t')
            print (line1.split(',M:')[1].split(',')[0] + '\n')
            statsFile.write(line1.split('C:')[1].split('[')[0] + '\t' + \
                            line1.split('[S:')[1].split(',')[0] + '\t' + \
                            line1.split('D:')[1].split(']')[0] + '\t' + \
                            line1.split(',F:')[1].split(',')[0] + '\t' + \
                            line1.split(',M:')[1].split(',')[0] + '\n')
    BUSCO_result.close()
    
sampleFile.close()
statsFile.close()

EOF